In [101]:
import argparse
import boto3
import paramiko
import os, sys, time
from pathlib import Path

In [ ]:
profile = [
    'region': 
]

In [5]:
session = boto3.Session()

In [170]:
ec2 = session.resource('ec2')

In [192]:
c = ec2.vpcs.filter(Filters=[{'Name': 'tag-value', 'Values': ['sdfjklsdf']}])

In [194]:
if list(c):
    print('sdjfklds')

In [186]:
list(ec2.vpcs.filter(Filters=[{'Name': 'tag-value', 'Values': [name]}]))[0].id

TypeError: 'ec2.vpcsCollection' object does not support indexing

In [182]:
list(ec2.vpcs.all())

[ec2.Vpc(id='vpc-6e6b2a17'),
 ec2.Vpc(id='vpc-bc7e3fc5'),
 ec2.Vpc(id='vpc-56b0f02f')]

In [173]:
ec2.vpcs(Filters=[{'Name': 'tag-value', 'Values': [name]}])

TypeError: 'ec2.vpcsCollectionManager' object is not callable

In [7]:
ec2_client = session.client('ec2')

In [39]:
ec2_client.describe_security_groups()

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 08:38:27 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'acf17ac6-1d70-4313-b783-c0146c637c92',
  'RetryAttempts': 0},
 'SecurityGroups': [{'Description': 'default group',
   'GroupId': 'sg-0b31743b',
   'GroupName': 'default',
   'IpPermissions': [{'FromPort': 0,
     'IpProtocol': 'tcp',
     'IpRanges': [],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': 65535,
     'UserIdGroupPairs': [{'GroupId': 'sg-0b31743b',
       'GroupName': 'default',
       'UserId': '698042698196'}]},
    {'FromPort': 0,
     'IpProtocol': 'udp',
     'IpRanges': [],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': 65535,
     'UserIdGroupPairs': [{'GroupId': 'sg-0b31743b',
       'GroupName': 'default',
       'UserId': '698042698196'}]},
    {'FromPort': -1,
     'IpProtocol':

In [270]:
def get_vpc(name):
    vpcs = list(ec2.vpcs.filter(Filters=[{'Name': 'tag-value', 'Values': [name]}]))
    return vpcs[0] if vpcs else None

In [271]:
def get_security_group(name, vpc_id):
    sg_tag = f'{name}-security-group'
    filters = [{'Name': 'group-name', 'Values': [sg_tag]}, {'Name': 'vpc-id', 'Values': [vpc_id]}]
    sgs = list(ec2.security_groups.filter(Filters=filters))
    return sgs[0] if sgs else None

In [272]:
def get_subnet(name, vpc_id):
    subnet_tag = f'{name}-subnet'
    filters = [{'Name': 'tag-value', 'Values': [subnet_tag]}, {'Name': 'vpc-id', 'Values': [vpc_id]}]
    subnets = list(ec2.subnets.filter(Filters=filters))
    return subnets[0] if subnets else None

In [302]:
list(vpc.subnets)

TypeError: 'ec2.Vpc.subnetsCollectionManager' object is not iterable

In [301]:
vpc.subnets

ec2.Vpc.subnetsCollectionManager(ec2.Vpc(id='vpc-f8bcfc81'), ec2.Subnet)

In [273]:
def get_vpc_ids(name):
    vpc = get_vpc(name)
    sg = get_security_group(name, vpc_id)
    subnet = get_subnet(name, vpc_id)
    return vpc.id, sg.id, subnet.id

In [274]:
name='fast-ai'

In [276]:
get_vpc_ids(name)

('vpc-6e6b2a17', 'sg-f60fca88', 'subnet-f056ff89')

### Creating a vpc

In [106]:
a = Path.home()

PosixPath('/Users/andrewshaw')

In [ ]:
ec2.create_key_pair()

In [263]:
def create_ec2_keypair(name):
    ssh_dir = Path.home()/'.ssh'
    (ssh_dir).mkdir(exist_ok=True)
    keypair_name = f'aws-key-{name}'
    filename = ssh_dir/f'{keypair_name}.pem'
    if filename.exists():
        print('Keypair exists')
        return
    keypair = ec2.create_key_pair(KeyName=keypair_name)
    keypair_out = keypair.key_material
    outfile = open(filename,'w')
    outfile.write(keypair_out)
    os.chmod(filename, 0o400)
    print('Created keypair')

In [257]:
# keypair = ec2.create_key_pair(KeyName='test-fast')

In [268]:
# ec2_client.delete_key_pair(KeyName='aws-key-test-fast')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '227',
   'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 18:54:54 GMT',
   'server': 'AmazonEC2'},
  'HTTPStatusCode': 200,
  'RequestId': 'c01140cc-baab-4628-9601-7dca6fc23be6',
  'RetryAttempts': 0}}

In [277]:
sg = get_security_group(name, vpc_id)

In [ ]:
sg.authorize_ingress

In [280]:
sg.authorize_ingress(IpProtocol='tcp', FromPort=8887, ToPort=8887, CidrIp=cidr)

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 18:59:37 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'e508054f-a2d4-4aa1-a3a1-57a392e606ef',
  'RetryAttempts': 0}}

In [281]:
def create_security_group(name, vpc):
    cidr = '0.0.0.0/0'
    sg = vpc.create_security_group(GroupName=f'{name}-security-group-test', Description='SG for {name} machine')
    # ssh
    sg.authorize_ingress(IpProtocol='tcp', FromPort=22, ToPort=22, CidrIp=cidr)
    # jupyter notebook
    sg.authorize_ingress(IpProtocol='tcp', FromPort=8888, ToPort=8898, CidrIp=cidr)
    return sg_id

In [ ]:
def create_vpc(name):
    cidr_block='10.0.0.0/28'
    vpc = ec2.create_vpc(CidrBlock=cidr_block)
    vpc.modify_attribute(EnableDnsSupport={'Value':True})
    vpc.modify_attribute(EnableDnsHostnames={'Value':True})
    vpc.create_tags(Tags=[{'Key':'Name','Value':name}])
    
    rt = vpc.create_route_table()
    
    
    return vpc

In [ ]:
def create_route_table()

In [292]:
subnet = vpc.create_subnet(CidrBlock=cidr_block)

In [294]:
subnet.create_tags(Tags=[{'Key':'Name','Value':f'{name}-subnet'}])

[ec2.Tag(resource_id='subnet-0fc10b44', key='Name', value='fast-ai-subnet')]

In [289]:
rt = vpc.create_route_table()

In [293]:
rt.create_tags(Tags=[{'Key':'Name','Value':f'{name}-route-table'}])

[ec2.Tag(resource_id='rtb-79dca001', key='Name', value='fast-ai-subnet')]

ec2.Vpc.subnetsCollectionManager(ec2.Vpc(id='vpc-f8bcfc81'), ec2.Subnet)

In [ ]:
rt.associate_with_subnet

In [ ]:

route_table = vpc.create_route_table()
route_table.create_tags(Tags=u.make_name(ROUTE_TABLE_NAME))

In [ ]:

export internetGatewayId=$(aws ec2 create-internet-gateway --query 'InternetGateway.InternetGatewayId' --output text)
aws ec2 create-tags --resources $internetGatewayId --tags --tags Key=Name,Value=$name-gateway
aws ec2 attach-internet-gateway --internet-gateway-id $internetGatewayId --vpc-id $vpcId

export subnetId=$(aws ec2 create-subnet --vpc-id $vpcId --cidr-block 10.0.0.0/28 --query 'Subnet.SubnetId' --output text)
# aws ec2 modify-subnet-attribute --subnet-id $subnetId --map-public-ip-on-launch // Public id needed?
aws ec2 create-tags --resources $subnetId --tags --tags Key=Name,Value=$name-subnet

export routeTableId=$(aws ec2 create-route-table --vpc-id $vpcId --query 'RouteTable.RouteTableId' --output text)
aws ec2 create-tags --resources $routeTableId --tags --tags Key=Name,Value=$name-route-table
export routeTableAssoc=$(aws ec2 associate-route-table --route-table-id $routeTableId --subnet-id $subnetId --output text)
aws ec2 create-route --route-table-id $routeTableId --destination-cidr-block 0.0.0.0/0 --gateway-id $internetGatewayId


In [ ]:
#!/bin/bash

# settings
export name="fast-ai"

export cidr="0.0.0.0/0"
export vpcTag=$name
export sgTag="SG for fast.ai machine"
export subnetTag="$name-subnet"

hash aws 2>/dev/null
if [ $? -ne 0 ]; then
    echo >&2 "'aws' command line tool required, but not installed.  Aborting."
    exit 1
fi

if [ -z "$(aws configure get aws_access_key_id)" ]; then
    echo "AWS credentials not configured.  Aborting"
    exit 1
fi

export vpcId=$(aws ec2 create-vpc --cidr-block 10.0.0.0/28 --query 'Vpc.VpcId' --output text)
aws ec2 create-tags --resources $vpcId --tags --tags Key=Name,Value=$name
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-support "{\"Value\":true}"
aws ec2 modify-vpc-attribute --vpc-id $vpcId --enable-dns-hostnames "{\"Value\":true}"

export internetGatewayId=$(aws ec2 create-internet-gateway --query 'InternetGateway.InternetGatewayId' --output text)
aws ec2 create-tags --resources $internetGatewayId --tags --tags Key=Name,Value=$name-gateway
aws ec2 attach-internet-gateway --internet-gateway-id $internetGatewayId --vpc-id $vpcId

export subnetId=$(aws ec2 create-subnet --vpc-id $vpcId --cidr-block 10.0.0.0/28 --query 'Subnet.SubnetId' --output text)
# aws ec2 modify-subnet-attribute --subnet-id $subnetId --map-public-ip-on-launch // Public id needed?
aws ec2 create-tags --resources $subnetId --tags --tags Key=Name,Value=$name-subnet

export routeTableId=$(aws ec2 create-route-table --vpc-id $vpcId --query 'RouteTable.RouteTableId' --output text)
aws ec2 create-tags --resources $routeTableId --tags --tags Key=Name,Value=$name-route-table
export routeTableAssoc=$(aws ec2 associate-route-table --route-table-id $routeTableId --subnet-id $subnetId --output text)
aws ec2 create-route --route-table-id $routeTableId --destination-cidr-block 0.0.0.0/0 --gateway-id $internetGatewayId

export securityGroupId=$(aws ec2 create-security-group --group-name $name-security-group --description $sgTag --vpc-id $vpcId --query 'GroupId' --output text)
# ssh
aws ec2 authorize-security-group-ingress --group-id $securityGroupId --protocol tcp --port 22 --cidr $cidr
# jupyter notebook
aws ec2 authorize-security-group-ingress --group-id $securityGroupId --protocol tcp --port 8888-8898 --cidr $cidr

if [ ! -d ~/.ssh ]
then
	mkdir ~/.ssh
fi

if [ ! -f ~/.ssh/aws-key-$name.pem ]
then
	aws ec2 create-key-pair --key-name aws-key-$name --query 'KeyMaterial' --output text > ~/.ssh/aws-key-$name.pem
	chmod 400 ~/.ssh/aws-key-$name.pem
fi